In [4]:
pip install jieba 

  Created wheel for jieba: filename=jieba-0.42.1-cp37-none-any.whl size=19314482 sha256=fda769ca33edd02fbe1f9fee4466a69a7c3f182416f1c0292bd1e119f5610ba5
  Stored in directory: C:\Users\liaoz\AppData\Local\pip\Cache\wheels\af\e4\8e\5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install translate

  Created wheel for distlib: filename=distlib-0.3.0-cp37-none-any.whl size=340433 sha256=4d567002a292c57aa53285b98bde41a41891d4d86127a393d5fdcfb023e0a45f
  Stored in directory: C:\Users\liaoz\AppData\Local\pip\Cache\wheels\6e\e8\db\c73dae4867666e89ba3cfbc4b5c092446f0e584eda6f409cbb
Successfully built distlib
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
Note: you may need to restart the kernel to use updated packages.


ERROR: astroid 2.3.1 requires typed-ast<1.5,>=1.4.0; implementation_name == "cpython" and python_version < "3.8", which is not installed.
ERROR: astroid 2.3.1 has requirement six==1.12, but you'll have six 1.14.0 which is incompatible.


In [122]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain
import random

In [123]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d

In [205]:
from translate import Translator
translator= Translator(from_lang="chinese",to_lang="english")

In [206]:
translation=translator.translate("今天")
print(translation)

Today


In [257]:
import jieba.posseg as peg
in_str='''我今天吃苹果了'''
words=peg.cut(in_str)
result1=" "
tags=" "
for word,flag in words:
   # temp="%s_%s "%(word,flag)

    translation=translator.translate(word)
    result1=result1+translation+" "
    tag="%s /"%flag
    tags=tags+tag
   # word_counts = pair_counts(tag,words)
print(result1)
print(tags)

AttributeError: 'generator' object has no attribute 'cut'

In [213]:
def read_data(filename):
    """Read tagged sentence data"""
    with open(filename, 'r',encoding="utf-8") as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

Sentence = namedtuple("Sentence", "words tags")

In [223]:
tagset = read_tags("data/brown/aa.txt")
sentences = read_data("data/brown/bb.txt")
sentences

OrderedDict([('\ufeff1',
              Sentence(words=('我们的', '班', '有', '一个', '很', '优秀的', '同学', '，', '她', '不光', '学习', '是', '我们', '班的', '第一名', '，', '而且', '在', '我们', '班', '还是', '经常', '受', '表扬的', '人物', '，', '她', '就是', '我们', '班的', '夏宇', '，', '她', '是', '一个', '特别', '优秀的', '学生', '，', '每年的', '三好', '学生', '，', '她', '还', '很', '热爱', '班', '集体', '。'), tags=('our', 'class', 'have', 'a', 'very', 'good', 'classmate', ',', 'she', 'not only', 'study', 'is', 'our', 'class', 'first-class', ',', 'and', 'in', 'our', 'class', 'is', 'usually', 'been', 'Praise', 'character', ',', 'she', 'is', 'our', 'class', 'xiayu', ',', 'she', 'is', 'a', 'very', 'good', 'student', ',', 'annual', 'good', 'student', ',', 'she', 'also', 'very', 'like', 'class', 'collective', '.'))),
             ('2',
              Sentence(words=('昨天', '放学的', '时候', '，', '同学们', '听到', '铃声', '，', '一窝蜂的', '都', '出去', '，', '她', '忽然', '停下', '脚步', '，', '我', '顺着', '她的', '目光', '，', '看见', '教室', '里', '非常', '杂乱', '，', '地上', '还有', '一大堆', '同学', '留下的', '纸屑', '

In [224]:
class Dataset(namedtuple("_Dataset", "sentences keys vocab X tagset Y training_set testing_set N stream")):
    def __new__(cls, tagfile, datafile, train_test_split=0.8, seed=112890):
        tagset = read_tags(tagfile)
        sentences = read_data(datafile)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.words for s in sentences.values()]))
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        N = sum(1 for _ in chain(*(s.words for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences, tagset,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())
    
    
class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].words for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

In [225]:
data = Dataset("data/brown/aa.txt", "data/brown/bb.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 12 sentences in the corpus.
There are 9 sentences in the training set.
There are 3 sentences in the testing set.


In [226]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 265 samples of 152 unique words in the corpus.
There are 159 samples of 103 unique words in the training set.
There are 106 samples of 59 unique words in the testing set.
There are 49 words in the test set that are missing in the training set.


In [227]:
key = '2'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: 2
words:
	('昨天', '放学的', '时候', '，', '同学们', '听到', '铃声', '，', '一窝蜂的', '都', '出去', '，', '她', '忽然', '停下', '脚步', '，', '我', '顺着', '她的', '目光', '，', '看见', '教室', '里', '非常', '杂乱', '，', '地上', '还有', '一大堆', '同学', '留下的', '纸屑', '。')
tags:
	('yesterday', 'after school', 'time', ',', 'student', 'hear', 'ring bell', ',', 'a lot', 'all', 'get out', ',', 'she', 'suddently', 'stop', 'pace', ',', 'I', 'along', 'hers', 'eye', ',', 'see', 'class', 'in', 'very', 'massly', ',', 'ground', 'also', 'a load of', 'student', 'left', 'paper', '.')


In [228]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('我们的', '班', '有', '一个', '很', '优秀的', '同学', '，', '她', '不光', '学习', '是', '我们', '班的', '第一名', '，', '而且', '在', '我们', '班', '还是', '经常', '受', '表扬的', '人物', '，', '她', '就是', '我们', '班的', '夏宇', '，', '她', '是', '一个', '特别', '优秀的', '学生', '，', '每年的', '三好', '学生', '，', '她', '还', '很', '热爱', '班', '集体', '。')

Labels 1: ('our', 'class', 'have', 'a', 'very', 'good', 'classmate', ',', 'she', 'not only', 'study', 'is', 'our', 'class', 'first-class', ',', 'and', 'in', 'our', 'class', 'is', 'usually', 'been', 'Praise', 'character', ',', 'she', 'is', 'our', 'class', 'xiayu', ',', 'she', 'is', 'a', 'very', 'good', 'student', ',', 'annual', 'good', 'student', ',', 'she', 'also', 'very', 'like', 'class', 'collective', '.')

Sentence 2: ('昨天', '放学的', '时候', '，', '同学们', '听到', '铃声', '，', '一窝蜂的', '都', '出去', '，', '她', '忽然', '停下', '脚步', '，', '我', '顺着', '她的', '目光', '，', '看见', '教室', '里', '非常', '杂乱', '，', '地上', '还有', '一大堆', '同学', '留下的', '纸屑', '。')

Labels 2: ('yesterday', 'after school', 'time', ',', 'student', 'hear'

In [229]:
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 3: break


Stream (word, tag) pairs:

	 ('我们的', 'our')
	 ('班', 'class')
	 ('有', 'have')
	 ('一个', 'a')
	 ('很', 'very')


In [230]:
words = [word for i, (word, tag) in enumerate(data.training_set.stream())]
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
words[0:4], tags[0:4]

(['昨天', '放学的', '时候', '，'], ['yesterday', 'after school', 'time', ','])

In [231]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d
        
word_counts = pair_counts(words, tags)

In [232]:
mfc_table = dict((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())

In [233]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d
        
word_counts = pair_counts(words, tags)

In [234]:
mfc_table = dict((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())

In [235]:
i = 0
for key, value in mfc_table.items():
    print(key, value)
    i += 1
    if i > 3: break

昨天 yesterday
放学的 after school
时候 time
， ,


In [236]:
FakeState = namedtuple('FakeState', 'name')

class MFCTagger:
    missing = FakeState(name = '<MISSING>')
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

In [237]:
mfc_model = MFCTagger(mfc_table)

In [238]:
def replace_unknown(sequence):
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]

In [239]:
data.testing_set.keys

['\ufeff1', '10', '9']

In [244]:
for key in data.testing_set.keys[1:]:
    print("Sentence Key: {}\n".format(key))
    print("Sentence: {}\n".format(data.sentences[key].words))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: 10

Sentence: ('三月', '到了', '，', '春', '姑娘', '也', '悄悄的', '来到了', '我们的', '身边', '，', '她', '轻手轻脚', '，', '悄悄的', '走着', '，', '浑身', '散发着', '清香', '，', '她', '羞涩地', '走着', '。')

Predicted labels:
-----------------
['<MISSING>', '<MISSING>', ',', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', ',', 'she', '<MISSING>', ',', '<MISSING>', '<MISSING>', ',', '<MISSING>', '<MISSING>', '<MISSING>', ',', 'she', '<MISSING>', '<MISSING>', '.']

Actual labels:
--------------
('march', 'arrived', ',', 'spring', 'girl', 'also', 'quietly', 'arrive', 'our', 'side', ',', 'she', 'lightly', ',', 'silent', 'walking', ',', 'whole body', 'exude', 'fragrance', ',', 'she', 'shyly', 'walking', '.')


Sentence Key: 9

Sentence: ('这', '就是', '我们', '班的', '优秀', '同学', ',', '我', '也要', '向', '她', '学习', '，', '不仅', '要', '好好', '学习', '，', '更要', '爱护', '我们的', '班', '集体', '，', '因为', '班', '集体', '就是', '我们的', '家', '。')

Predicted labels:
-----------------
['<MISSING>', '<MISSING>', 'we',

In [245]:
def accuracy(X, Y, model):
    
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [246]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

training accuracy mfc_model: 93.71%
testing accuracy mfc_model: 29.25%
